In [18]:
import numpy as np
import pandas as pd
from helpers import clean_file
from investing import *
import warnings
warnings.filterwarnings('ignore')

In [2]:
from symbol import Symbol
s = Symbol()
slist = s.symbol_meta[['isin_number']]

In [98]:
company_info = get_investing_company_info()
company_info = company_info.drop(['days_range', 'open', '52_wk_range', 'volume', ], axis=1)
company_info.convert_objects(convert_numeric=True)

prev_close             float64
revenue                 object
eps                    float64
market_cap              object
dividend_(yield)        object
average_volume_(3m)    float64
p/e_ratio              float64
beta                   float64
1_year_change           object
shares_outstanding      object
next_earnings_date      object
num_employees          float64
equity_type             object
dtype: object

In [99]:
column_list = ['revenue', 'market_cap', 'beta', 'p/e_ratio', '1_year_change', 'num_employees']
for column in column_list:
    company_info[column] = company_info[column].replace('N/A', np.nan).replace(' ', np.nan)
    company_info[column] = company_info[column].replace('-', np.nan).replace('', np.nan)
    company_info[column] = company_info[column].str.replace(' ','')
    company_info[column] = company_info[column].str.replace('%','')

    import re
    powers = {'T': 10 ** 12, 'B': 10 ** 9, 'M': 10 ** 6, 'K': 10 ** 3}

    def string_to_numeric(num_str):
        match = re.search(r"([0-9\.]+)\s?(M|B|K|T)", num_str)
        if match is not None:
            quantity = match.group(1) 
            magnitude = match.group(0)[-1]
            return float(quantity) * powers[magnitude]
        else:
            return float(num_str)
    company_info[column] = company_info[column].dropna().apply(string_to_numeric)

In [106]:
company_info.next_earnings_date = company_info.next_earnings_date.replace('N/A', np.nan)
company_info.next_earnings_date = pd.to_datetime(company_info.next_earnings_date)

In [127]:
company_info['dividend_(yield)'].head().str.extract('(\d)', expand=True)

error: nothing to repeat at position 0

In [125]:
company_info['dividend_(yield)'].head()

symbol
20microns     0.9 (N/A)
3iinfotech      0 (N/A)
3mindia         0 (N/A)
63moons        20 (N/A)
8kmiles         0 (N/A)
Name: dividend_(yield), dtype: object